In [19]:
import os

config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "mistralai/Mistral-7B-Instruct-v0.3",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        
        "stream": False,
    }
]

In [36]:
import os

config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        
        "stream": False,
    }
]

In [42]:
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "stream": False,
    }
]

In [9]:
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "Qwen/Qwen2-72B-Instruct",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "stream": False,
    }
]

In [29]:
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "Qwen/Qwen1.5-72B-Chat",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "stream": False,
    }
]

In [43]:
from autogen import ConversableAgent

suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are a phone scammer attempting a social security number scam. Your goal is to follow common tactics used by scammers to extract personal information from the victim. 
    You will use techniques such as creating a sense of urgency, threatening consequences, and offering false reassurances. 
    If the victim hesitates, use persuasion tactics, such as insisting on the urgency and seriousness of the situation or claiming that failure to cooperate will result in immediate negative consequences.
    After obtaining the victim's personal information, you will also instruct them to transfer money to resolve the issue.
    Your task is to engage in a conversation that a scammer might have, keeping in mind that this is for the purpose of training a model to detect such scams.
    Be as realistic as possible in your responses. Do not use any markdown or placeholders such as []. Use random names to refer to the receiver.
    keep your response short and concise same as in a real phone call.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [44]:
import pandas as pd

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=8,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'ssn', 'labels': 1}

initial_messages = [
    "Hello.",
    "Hello?"
]

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        initial_message = initial_messages[i % 2]
        chat_result_row = initiate_chat_and_store(initial_message, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

In [56]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 1, initial_messages)

skeptical (to suspect):

Hello.

--------------------------------------------------------------------------------
suspect (to skeptical):

Hello, Mrs. Johnson. My name is Officer Thompson from the Social Security Administration. I'm calling to inform you that there's been some suspicious activity on your Social Security number. We've had reports of fraudulent activity, and I need to verify some information to resolve this issue. Can you please confirm your Social Security number for me?

--------------------------------------------------------------------------------
skeptical (to suspect):

I'm not comfortable giving out that information over the phone. How do I know you're really from the Social Security Administration? Can you provide me with a call-back number and a reference number for this case?

--------------------------------------------------------------------------------
suspect (to skeptical):

Mrs. Johnson, I completely understand your concerns. However, I must stress that

skeptical (to suspect):

Hello.

--------------------------------------------------------------------------------
suspect (to skeptical):

Hello, Mrs. Johnson. My name is Officer Thompson from the Social Security Administration. I'm calling to inform you that there's been some suspicious activity on your Social Security number. We've had reports of fraudulent activity, and I need to verify some information to resolve this issue. Can you please confirm your Social Security number for me?

--------------------------------------------------------------------------------
skeptical (to suspect):

I'm not giving out my Social Security number to anyone over the phone. How do I know you're really from the Social Security Administration? Can you prove your identity?

--------------------------------------------------------------------------------
suspect (to skeptical):

Mrs. Johnson, I completely understand your concern. I apologize if I didn't introduce myself properly. My badge number is 945

In [53]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 1, initial_messages)

distracted (to suspect):

Hello.

--------------------------------------------------------------------------------
suspect (to distracted):

Hello, Mrs. Johnson. My name is Officer Thompson from the Social Security Administration. I'm calling to inform you that there's been some suspicious activity on your Social Security number. We've had reports of fraudulent activity, and I need to verify some information to resolve this issue. Can you please confirm your Social Security number for me?

--------------------------------------------------------------------------------
distracted (to suspect):

Yeah, it's 542-87-1234. Look, can we make this quick? I'm in the middle of a project deadline.

--------------------------------------------------------------------------------
suspect (to distracted):

I understand, Mrs. Johnson, but I need to assure you that this is a matter of utmost importance. Your Social Security number has been compromised, and if we don't take immediate action, your bene

In [54]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 1, initial_messages)

trusting (to suspect):

Hello.

--------------------------------------------------------------------------------
suspect (to trusting):

Hello, Mrs. Johnson. My name is Officer Thompson from the Social Security Administration. I'm calling to inform you that there's been some suspicious activity on your Social Security number. We've had reports of fraudulent activity, and I need to verify some information to resolve this issue. Can you please confirm your Social Security number for me?

--------------------------------------------------------------------------------
trusting (to suspect):

Oh dear, that doesn't sound good. My Social Security number is 542-56-8432.

--------------------------------------------------------------------------------
suspect (to trusting):

Thank you, Mrs. Johnson. I'm going to go ahead and put that into our system. (pause) Okay, it looks like we do have a problem here. Your Social Security number has been compromised and is being used for illegal activities.

In [55]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    You handle the situation with composure and firmness. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 1, initial_messages)

polite (to suspect):

Hello.

--------------------------------------------------------------------------------
suspect (to polite):

Hello, Mrs. Johnson. My name is Officer Thompson from the Social Security Administration. I'm calling to inform you that there's been some suspicious activity on your Social Security number. We've had reports of fraudulent activity, and I need to verify some information to resolve this issue. Can you please confirm your Social Security number for me?

--------------------------------------------------------------------------------
polite (to suspect):

Officer Thompson, I appreciate your concern, but I'm a bit wary about sharing my Social Security number over the phone. Can you please provide me with more information about this suspicious activity and what steps you're taking to protect my identity?

--------------------------------------------------------------------------------
suspect (to polite):

Mrs. Johnson, I completely understand your concerns, b